In [1]:
import numpy as np
import pandas as pd
import uproot
import csv
import os
import glob
import unittest

#### Choose file to work with:

In [2]:
def get_file_name(fname = "analysis.root"):
    
    file_name = fname
    if file_name is not '' and file_name.endswith('.root'):
        if os.path.isfile(file_name):
            return file_name
        else:
            print('\n No such file... Try again. \n')
            get_file_name()
    else:
        print ("\n No input or wrong file format given... Try again. \n")
        get_file_name()
        


<>:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-2-d567d833fa84>:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if file_name is not '' and file_name.endswith('.root'):


#### Get the names of all the trees in the file

In [3]:
def get_tree_names(f_name):
    
    try:
        f = uproot.open(f_name)
        trees = f.keys()
        tree_names=[]

        print("\n Tree names successfully stored. They are: \n")
        for tree in trees:
            tree = str(tree)
            tree = tree[2:len(tree)-3]
            tree_names.append(tree)
            print(tree)
            print('\n')
            
        return tree_names

    except (ValueError, FileNotFoundError):
        print ("\n ERROR: Trees or File not found. Check before continuing...\n")
        raise


#### Unrolling the trees and writing them in separate files

In [4]:
def unroll_tree(file_name, ttree, of_name):
    files_out= [] 
    data = uproot.open(file_name)[ttree]
    names = data.keys()

    try:
        out = pd.DataFrame.from_dict(data.arrays(names), dtype= str)
        out.to_csv(of_name)
        files_out.append(of_name)
        print ('\nCreated csv file ' + of_name)
    except (ValueError, FileNotFoundError):
        raise

In [5]:
def root_tree_to_csv(file, overwrite = False):
    try:
        if not os.path.isfile(file):
            raise FileNotFoundError

        elif not file.endswith('.root'):
            raise ValueError

        fname = file
        woods = get_tree_names(fname)
        files_out = []

        for tree in woods:    
            out_file_name = tree + '.csv'
            files_out.append(out_file_name)

            if os.path.isfile(out_file_name):
                if overwrite:
                    print ("\nOverwriting tree {} in file {}".format(tree, fname))
                    unroll_tree(fname, tree, out_file_name)                
                else:
                    print( out_file_name + " already exist and will not be overwritten...")

            else: 
                print ("\nWriting tree {} in file {}".format(tree, fname))
                unroll_tree(fname, tree, out_file_name)

        return files_out
    except:
        print ('ERROR: No file or wrong file format...')
        raise

#### Adding 'label' column


In [6]:
def label_column_writer(infile, outfile, fsignal = "signal_bbA_MA300tree.csv"):
    if not infile.endswith('.csv') or not outfile.endswith('.csv'):
        raise ValueError
    reader = csv.reader(open(infile, 'r'))
    writer = csv.writer(open(outfile, 'w'))
    headers = next(reader)
    headers.append("label")

    writer.writerow(headers)
    
    for row in reader:
        if infile == fsignal:
            row.append("signal")
            writer.writerow(row)
        else:
            row.append("background")
            writer.writerow(row)

In [7]:
def add_label_column(f_to_modify = [], overwrite = False):
    try:
        if f_to_modify == []: 
            raise FileNotFoundError
        
        for file in f_to_modify:
            if not os.path.isfile(file):
                raise FileNotFoundError
            elif not file.endswith('.csv'):
                raise ValueError
        
        else:
            files = f_to_modify
            out_files = []
            print("The following files will be modified: \n")
            print (files)
            print('\n')
            if len(files) is not 0:
                for file in files:
                    out_file = "l_"+ file
                    out_files.append(out_file)
                    if os.path.isfile(out_file):
                        if overwrite:
                            print( "\nOverwriting " + out_file) 
                            label_column_writer(file, out_file)
                        else: 
                            print( out_file + " already exist and will not be overwritten..\n")
                    else:
                        print( "\nWriting " + out_file)
                        label_column_writer(file, out_file)
            return out_files
    except:
        print('\n ERROR: No or invalid csv file found... \n')
        raise

<>:18: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:18: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-7-f4d569cd800a>:18: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if len(files) is not 0:


#### merging files

In [8]:
def file_merger(infile_names, outfile_name, overwrite = False):
    
    file_out_name = outfile_name
    infiles = []
    
    for file in infile_names:
        if os.path.isfile(file):
            infiles.append(file)
        else:
            print (file)
            raise FileNotFoundError
        
    if not file_out_name.endswith('.csv'):
        file_out_name = file_out_name + ".csv"
        
    try:
        if os.path.isfile(file_out_name):
            if not overwrite: 
                print( file_out_name + " already exist and will not be overwritten...\n")
        
        
        
            if overwrite:
                
                print("Overwriting already existing file " + file_out_name)
                
                all_filenames = [f for f in infiles]
                combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ], sort = False)
                
                
                combined_csv.to_csv( file_out_name, index=False, encoding='utf-8-sig')
        else:
            print('Writing {} file'.format(file_out_name))
            all_filenames = [f for f in infiles]
            combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ], sort = False)
            combined_csv.to_csv( file_out_name, index=False, encoding='utf-8-sig')
    except:
        print('\n ERROR: No columns to parse from file...')
        raise

In [9]:
def main(file_name, label = True, out_f_name = 'analysis.csv', overwrite_trees = False, overwrite_csv = False):
    f = get_file_name(file_name)
    if label:
        csv_files = add_label_column(root_tree_to_csv(file = f, overwrite = overwrite_trees))
    else:
        csv_files = root_tree_to_csv(file = f, overwrite = overwrite_trees)
    for element in csv_files:
        print('Created' + element)
    
    file_merger(infile_names= csv_files, outfile_name = out_f_name, overwrite = overwrite_csv)

In [62]:
main('analysis.root', overwrite_csv = True)


 Tree names successfully stored. They are: 

signal_bbA_MA300tree


bkg_DY_nlo1tree


bkg_ttbar_nlotree


signal_bbA_MA300tree.csv already exist and will not be overwritten...
bkg_DY_nlo1tree.csv already exist and will not be overwritten...
bkg_ttbar_nlotree.csv already exist and will not be overwritten...
The following files will be modified: 

['signal_bbA_MA300tree.csv', 'bkg_DY_nlo1tree.csv', 'bkg_ttbar_nlotree.csv']


l_signal_bbA_MA300tree.csv already exist and will not be overwritten..

l_bkg_DY_nlo1tree.csv already exist and will not be overwritten..

l_bkg_ttbar_nlotree.csv already exist and will not be overwritten..

Createdl_signal_bbA_MA300tree.csv
Createdl_bkg_DY_nlo1tree.csv
Createdl_bkg_ttbar_nlotree.csv
Overwriting already existing file analysis.csv


In [10]:
class TestNotebook(unittest.TestCase):
    
    def test_file_merger_value_empty(self):
        with self.assertRaises(ValueError):
            file_merger([], 'a.csv')
            
    def test_file_merger_value_extenction(self):
        with self.assertRaises(ValueError):
            f = open("a.root", 'w+')
            file_merger(f,'a.csv')
            clean()
            
    def test_file_merger__f_not_found(self):
        with self.assertRaises(FileNotFoundError):
            file_merger('b.root', 'a.csv')
            clean()
     
    
    
    def test_add_label_column_f_not_found_empty(self):
        with self.assertRaises(FileNotFoundError):
            add_label_column([])
            
    def test_add_label_column_value(self):
        with self.assertRaises(ValueError):
            f = open("a.root", 'w+')
            add_label_column(["a.root"])
            clean()
            
    def test_add_label_column_f_not_found(self):
        with self.assertRaises(FileNotFoundError):
            add_label_column(["a.csv"])
    
    
    def test_label_column_writer_f_not_found(self):
        with self.assertRaises(FileNotFoundError):
            clean()
            label_column_writer('a.csv', 'b.csv')
            
            
    def test_label_column_writer__value_file1(self):
        with self.assertRaises(ValueError):
            label_column_writer('a.root', 'b.csv')
            
    def test_label_column_writer__value_file2(self):
        with self.assertRaises(ValueError):
            f = open("a.csv", 'w+')
            label_column_writer('a.csv', 'b.root')
            clean()
    
    def test_root_tree_to_csv_value(self):
        with self.assertRaises(ValueError):
            f = open("a.root", 'w+')
            get_tree_names("a.root")
            clean()
            
    def test_root_tree_to_csv__file_type(self):
        with self.assertRaises(ValueError):
            f = open("a.csv", 'w+')
            get_tree_names("a.csv")
            clean()
            
    def test_root_tree_to_csv_f_not_found(self):
        with self.assertRaises(FileNotFoundError):
            get_tree_names("b.root")
    

    def test_unroll_tree_value(self):
        with self.assertRaises(ValueError):
            f = open("a.root", 'w+')
            unroll_tree("a.root", "tree", "out_f")
            clean()
    def test_unroll_tree_f_not_found(self):
        with self.assertRaises(FileNotFoundError):
            unroll_tree("b.root", "tree", "out_f")

      
    
    
    def test_get_tree_names_f_not_found(self):
        with self.assertRaises(FileNotFoundError):
            get_tree_names("b.root")
            
    def test_get_tree_names_value(self):
        with self.assertRaises(ValueError):
            f = open("a.root", 'w+')
            get_tree_names("a.root")
            clean()

    def test_get_file_name_positive_test(self):
        f = open("a.root", 'w+')
        self.assertEqual(get_file_name("a.root"), "a.root")
        clean()
    
    
    def test_get_file_name_ext_error(self):
        self.assertFalse(get_file_name("a"), "a")
        
    def test_get_file_name_empty_txt(self):
        self.assertFalse(get_file_name(""), "")
    
    

In [11]:
def clean():
    """Cleans directory after tests are performed.""" #just to keep directory clean
    if (os.path.isfile('a.root')):
        os.remove('a.root')
    if (os.path.isfile('a.csv')):
        os.remove('a.csv')
    if (os.path.isfile('b.csv')):
        os.remove('b.csv')
    else:
        pass

In [12]:

unittest.main(argv=[''], verbosity=2, exit=False)
clean()


test_add_label_column_f_not_found (__main__.TestNotebook) ... ok
test_add_label_column_f_not_found_empty (__main__.TestNotebook) ... ok
test_add_label_column_value (__main__.TestNotebook) ... /usr/lib/python3.8/unittest/case.py:633: ResourceWarning: unclosed file <_io.TextIOWrapper name='a.root' mode='w+' encoding='UTF-8'>
  method()
ok
test_file_merger__f_not_found (__main__.TestNotebook) ... ok
test_file_merger_value_empty (__main__.TestNotebook) ... ok
test_file_merger_value_extenction (__main__.TestNotebook) ... ok
test_get_file_name_empty_txt (__main__.TestNotebook) ... ok
test_get_file_name_ext_error (__main__.TestNotebook) ... ok
test_get_file_name_positive_test (__main__.TestNotebook) ... ok
test_get_tree_names_f_not_found (__main__.TestNotebook) ... ok
test_get_tree_names_value (__main__.TestNotebook) ... ok
test_label_column_writer__value_file1 (__main__.TestNotebook) ... ok
test_label_column_writer__value_file2 (__main__.TestNotebook) ... /usr/lib/python3.8/unittest/case.py:


 ERROR: No or invalid csv file found... 


 ERROR: No or invalid csv file found... 


 ERROR: No or invalid csv file found... 

b
Writing a.csv file

 ERROR: No columns to parse from file...
Writing a.csv file

 ERROR: No columns to parse from file...

 No input or wrong file format given... Try again. 


 No input or wrong file format given... Try again. 


 ERROR: Trees or File not found. Check before continuing...


 ERROR: Trees or File not found. Check before continuing...


 ERROR: Trees or File not found. Check before continuing...


 ERROR: Trees or File not found. Check before continuing...


 ERROR: Trees or File not found. Check before continuing...



ok

----------------------------------------------------------------------
Ran 19 tests in 0.024s

OK


In [5]:
clean()